In [ ]:
import sympy.interactive
sympy.interactive.init_printing('mathjax')

In [ ]:
import sys
sys.path.append('/home/bnijholt/orbitalfield/')

In [ ]:
%matplotlib inline

# 1. Standard library imports
import os.path
from math import pi
from itertools import product
# 2. External package imports
import kwant
import types
import sympy
from sympy.physics.quantum import TensorProduct as kr
import numpy as np
import holoviews as hv
import scipy.sparse.linalg as sla
from scipy.constants import hbar, m_e, eV, physical_constants
# 3. Internal imports
from discretizer import Discretizer, momentum_operators

In [ ]:
sx, sy, sz = [sympy.physics.matrices.msigma(i) for i in range(1, 4)]
s0 = sympy.eye(2)
s0sz = np.kron(s0, sz)
s0s0 = np.kron(s0, s0)

# Parameters taken from arXiv:1204.2792
# All constant parameters, mostly fundamental constants, in a SimpleNamespace.
constants = types.SimpleNamespace(
    m=0.015 * m_e,  # effective mass in kg
    a=10,  # lattice spacing in nm
    g=50,  # Lande factor
    hbar=hbar,
    m_e=m_e,
    e=eV,
    eV=eV,
    meV=eV * 1e-3)

constants.t = (hbar ** 2 / (2 * constants.m)) * (1e18 / constants.meV)  # meV * nm^2
constants.mu_B = physical_constants['Bohr magneton'][0] / constants.meV

# All frequently used dimensions in a SimpleNamespace.
dimensions = types.SimpleNamespace(
    B=hv.Dimension(name=('B', r'$B$'), unit='T'),
    mu=hv.Dimension(name=('mu', r'$\mu$'), unit='meV'),
    gap=hv.Dimension(name='Band gap', unit=r'$\mu$eV'),
    # gap=hv.Dimension(name=r'$E_\textrm{gap}$', unit=r'\textmu eV'),
    decay_length=hv.Dimension(name='Inverse decay length', unit=r'$\mu m^{-1}$'),
    # decay_length=hv.Dimension(r'$\xi^{-1}$', unit=r'\textmu m$^{-1}$'),
    k=hv.Dimension(name=r'$k$', unit=r'nm$^{-1}$'),
    E=hv.Dimension(name=r'$E$', unit='meV'),
    delta=hv.Dimension(name=('Delta', r'$\Delta$'), unit='mev'),
    delta_ind=hv.Dimension(name=('Delta_ind', r'$\Delta_{ind}$'), unit='mev'),
    angles={'kdims': [hv.Dimension(name=('theta', r'$\theta$'), unit=r'$\pi$'),
                      hv.Dimension(name=('phi', r'$\phi$'), unit=r'$\pi$')]})

from types import SimpleNamespace 
def make_params(alpha=20,
                B_x=0,
                B_y=0,
                B_z=0,
                Delta=0.25,
                mu=0,
                orbital=True,
                A_correction=True,
                t=constants.t,
                g=constants.g,
                mu_B=constants.mu_B,
                V=lambda x,y,z: 0,
                **kwargs):
    """Function that creates a namespace with parameters.

    Parameters:
    -----------
    alpha : float
        Spin-orbit coupling strength in units of meV*nm.
    B_x, B_y, B_z : float
        The magnetic field strength in the x, y and z direction in units of Tesla.
    Delta : float
        The superconducting gap in units of meV.
    mu : float
        The chemical potential in units of meV.
    orbital : bool
        Switches the orbital effects on and off.
    A_correction : bool
        Corrects for the net supercurrent flowing in the wire. If True, the
        current will be set to zero.
    t : float
        Hopping parameter in meV * nm^2.
    g : float
        Lande g factor.
    mu_B : float
        Bohr magneton in meV/K.
    V : function
        Function of spatial coordinates (x, y, z) with is added to mu.

    Returns:
    --------
    p : types.SimpleNamespace object
        A simple container that is used to store Hamiltonian parameters.
    """
    p = types.SimpleNamespace(t=t,
                              g=g,
                              mu_B=mu_B,
                              alpha=alpha,
                              B_x=B_x,
                              B_y=B_y,
                              B_z=B_z,
                              Delta=Delta,
                              mu=mu,
                              orbital=orbital,
                              A_correction=A_correction,
                              V=V,
                              **kwargs)
    return p


In [ ]:
def cylinder_sector(r1, r2=0, phi=360, angle=0):
    phi *= np.pi / 360
    angle *= np.pi / 180
    r1sq, r2sq = r1 ** 2, r2 ** 2
    def sector(pos):
        x, y, z = pos
        n = (y + 1j * z) * np.exp(1j * angle)
        y, z = n.real, n.imag
        rsq = y ** 2 + z ** 2
        return r2sq <= rsq < r1sq and z >= np.cos(phi) * np.sqrt(rsq)
    r_mid = (r1 + r2) / 2
    return sector, (0, r_mid * np.sin(angle), r_mid * np.cos(angle))

r1 = 50
r2 = 70
phi = 135
angle = 0

In [ ]:
k_x, k_y, k_z = momentum_operators
t, B_x, B_y, B_z, mu_B, Delta, mu, alpha, g, V = sympy.symbols('t B_x B_y B_z mu_B Delta mu alpha g V', real=True)
t_interface = sympy.symbols('t_interface', real=True)
k =  sympy.sqrt(k_x**2+k_y**2+k_z**2)
a = 10
lat_e = kwant.lattice.general(a * np.eye(3), name='e')
lat_h = kwant.lattice.general(a * np.eye(3), name='h')

kin = (t * k**2 - mu - V) * s0
Rashba_SO = alpha * (k_y * sx - k_x * sy)
Zeeman = 0.5 * g * mu_B * (B_x * sx + B_y * sy + B_z * sz)
ham_e = kin + Rashba_SO + Zeeman
ham_h = -kin - Rashba_SO + Zeeman

args = dict(space_dependent={'V'}, lattice_constant=a, discrete_coordinates={'x', 'y', 'z'})

tb_normal_e = Discretizer(ham_e, **args)
tb_normal_h = Discretizer(ham_h, **args)
tb_interface_e = Discretizer(ham_e.subs(t, t_interface), **args)
tb_interface_h = Discretizer(ham_h.subs(t, t_interface), **args)

# syst = kwant.Builder()
syst = kwant.Builder(kwant.TranslationalSymmetry((-a, 0, 0)))
shape_normal = cylinder_sector(r1=r1, angle=angle)
shape_sc = cylinder_sector(r1=r2, r2=r1, phi=phi, angle=angle)
sc_sites = list(syst.expand(lat_e.shape(*shape_sc)))

syst[lat_e.shape(*shape_normal)] = tb_normal_e.onsite
syst[lat_h.shape(*shape_normal)] = tb_normal_h.onsite
syst[lat_e.shape(*shape_sc)] = tb_normal_e.onsite
syst[lat_h.shape(*shape_sc)] = tb_normal_h.onsite


def peierls(val, ind, electron=True):
    def phase(s1, s2, p):
        x, y, z = s1.pos
        A = lambda p, x, y, z: [p.B_y * z - p.B_z * y, 0, p.B_x * y]
        A_site = A(p, x, y, z)[ind]
        if p.A_correction:
            A_sc = [A(p, *site.pos) for site in sc_sites]
            A_site -= np.mean(A_sc, axis=0)[ind]
        A_site *= a * 1e-18 * eV / hbar
        
        if not electron:
            A_site *= -1
        
        return np.exp(-1j * A_site) * np.eye(2)

    def with_phase(s1, s2, p):
        if p.orbital:
            return phase(s1, s2, p).dot(val(s1, s2, p))
        else:
            return val(s1, s2, p)
    return with_phase


for tb, lat, electron in zip([tb_normal_e, tb_normal_h], [lat_e, lat_h], [True, False]):
    for hop, val in tb.hoppings.items():
        ind = np.argmax(hop.delta)
        syst[kwant.builder.HoppingKind(hop.delta, lat, lat)] = peierls(val, ind, electron)

def at_interface(site1, site2):
    return ((shape_sc[0](site1.pos) and shape_normal[0](site2.pos)) or
            (shape_normal[0](site1.pos) and shape_sc[0](site2.pos)))

# Hoppings at the barrier between wire and superconductor
for tb_interface, lat, electron in zip([tb_interface_e, tb_interface_h], [lat_e, lat_h], [True, False]):
    for hop, val in tb_interface.hoppings.items():
        hopping_iterator = ((i, j) for (i, j) in kwant.builder.HoppingKind(hop.delta, lat)(syst) if at_interface(i, j))
        ind = np.argmax(hop.delta)
        syst[hopping_iterator] = peierls(val, ind, electron)

hopping_iterator = [i for i in kwant.builder.HoppingKind((0, 0, 0), lat_e, lat_h)(syst) if shape_sc[0](i[0].pos)]
syst[hopping_iterator] = lambda site1, site2, p: p.Delta * np.eye(2)

fsyst = syst.finalized()

In [ ]:
kwant.plot(syst)

In [ ]:
%matplotlib inline
p = make_params(t_interface=7*constants.t/8, orbital=True, alpha=1000, Delta=60, mu_B=1, A_correction=False, B_x=1, B_y=1)
b1 = kwant.physics.Bands(fsyst, args=[p])
from fun import make_3d_wire_external_sc
b2 = kwant.physics.Bands(make_3d_wire_external_sc(angle=0), args=[p])
np.allclose(b1(1)[0], b2(1)[0]), (b1(1)[0] - b2(1)[0])